# How to build a network?
- Social network simulation?

- We have:
    - Datetime
    - Innovation
    - Location
- How to build a Network from this?
- -> Tree shaped?

# CSV Import
- Dask only import if one of the Inno Columns is ==True; 
- Loop trough all CSV's and append "tures" to a single dd
- Turn into gpd

In [1]:
import dask.dataframe as dd
import geopandas as gpd
import pandas as pd
import os
# Treat '' and np.inf as NaN
pd.options.mode.use_inf_as_na = True

ERROR 1: PROJ: proj_create_from_database: Open of /home/linusrg/Code/LILI/.micromamba/envs/LILI-Env/share/proj failed


In [2]:
path = '~/Code/LILI/data/' 
cols = ["inno_01","inno_02","inno_03","inno_04","inno_05","inno_06","inno_07","inno_08","inno_09","inno_10","inno_12","inno_13","inno_15","inno_16","inno_17","inno_18","inno_19","inno_20","inno_21","inno_22","inno_23","inno_24","inno_25","inno_26","inno_27","inno_29","inno_30","inno_31","inno_32","inno_33","inno_34","inno_39"]
imp_cols = ["twitter_id","user_id","user_location","longitude","latitude","place_box_1_long","place_box_1_lat","place_box_2_long","place_box_2_lat","country","country_code","place_full_name","place_id","place_name"] + cols
#empty result df
df_allgeo = pd.DataFrame()

In [3]:
def dask_imp(path, imp_cols=imp_cols):
        df = dd.read_csv(path, sep = '\t', lineterminator='\n', usecols=imp_cols, false_values =['', ' ', "''", "' '", "", " "], dtype=
        {'twitter_id': 'int64',
        'user_id': 'int64',
        'user_location': 'string',
        'longitude': 'float',
        'latitude': 'float',
        'place_box_1_long': 'string',
        'place_box_1_lat': 'string',
        'place_box_2_long': 'string',
        'place_box_2_lat': 'string',
        'country': 'string',
        'country_code': 'string',
        'place_full_name': 'string',
        'place_id': 'string',
        'place_name': 'string',
        'place_type': 'string',
        'inno_01': 'boolean',
        'inno_02': 'boolean',
        'inno_03': 'boolean',
        'inno_04': 'boolean',
        'inno_05': 'boolean',
        'inno_06': 'boolean',
        'inno_07': 'boolean',
        'inno_08': 'boolean',
        'inno_09': 'boolean',
        'inno_10': 'boolean',
        'inno_12': 'boolean',
        'inno_13': 'boolean',
        'inno_15': 'boolean',
        'inno_16': 'boolean',
        'inno_17': 'boolean',
        'inno_18': 'boolean',
        'inno_19': 'boolean',
        'inno_20': 'boolean',
        'inno_21': 'boolean',
        'inno_22': 'boolean',
        'inno_23': 'boolean',
        'inno_24': 'boolean',
        'inno_25': 'boolean',
        'inno_26': 'boolean',
        'inno_27': 'boolean',
        'inno_29': 'boolean',
        'inno_30': 'boolean',
        'inno_31': 'boolean',
        'inno_32': 'boolean',
        'inno_33': 'boolean',
        'inno_34': 'boolean',
        'inno_39': 'boolean'})
        return df        

In [4]:
#create subfolder list
subfolders = []
for root, dirs, files in os.walk('../data'):
   for name in files:
      if root in subfolders:
         pass
      else:
         subfolders.append(root) 
subfolders.sort()

In [5]:
#Create empty csv with actual headers to initialize dask data frame
# The empty.csv is included in the repo (unlike the data folder)
#df_s = df_i.head(0)
#df_s.to_csv('empty.csv', sep = "\t")


In [6]:
df_s = dask_imp('empty.csv')
df_s.shape[1]

46

In [7]:
#loop trough folders, parallel import in folders (parallelization across subfolders not possible with Dask, takes ~20s)
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
#ignore the warning, append still works and concat is not implemented in dask yet :)
df_s = dask_imp('empty.csv')
for folder in subfolders:
      df = dask_imp(f'{folder}/*.csv')
      
     # df[cols] = df[cols].applymap(lambda x: eval(x), meta = df[cols])
      df['innovation'] = df[cols].sum(axis=1)
      df_i = df[df.innovation != 0]
      df_i = df_i.drop('innovation', axis=1)

      if df_i.shape[1] == df_s.shape[1]:
           df_s = df_s.append(df_i)
      else: print(folder, "--> wrong column count")

In [8]:
### Empty or empty string in Inno to FALSE
# Replace NaN with 0

df_s[cols] = df_s[cols].replace(['', pd.NA, ' ', 'NaN', '<NA>', 'False'], 0)
df_s = df_s.replace('Fal', 0)

In [9]:
# Stack all the bool to one named column
df_s['inno_nr'] = df_s[cols].idxmax(1).astype('string')
df_n = df_s.drop(cols, axis=1)
df_n['inno_nr'] = df_n['inno_nr'].astype('string')

In [10]:
df_n.head(5)

/home/linusrg/Code/LILI/.micromamba/envs/LILI-Env/lib/python3.10/site-packages/dask/dataframe/core.py:7296: UserWarning: Insufficient elements for `head`. 5 elements requested, only 2 elements available. Try passing larger `npartitions` to `head`.
  warnings.warn(


,twitter_id,user_id,user_location,longitude,latitude,place_box_1_long,place_box_1_lat,place_box_2_long,place_box_2_lat,country,country_code,place_full_name,place_id,place_name,inno_nr
0,153387731521773569,191216844,Paraguay,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,inno_02
1,153525325689655297,30079977,Perú ツ,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,inno_02


In [11]:
# Create an export parquet
#df_n.to_parquet('all_raw_dask.parquet')

In [12]:
#turn dask data frame into pandas data frame
#this is a longish process. ~50min
df = df_n.compute()
df.to_parquet('all2.parquet')

ValueError: Fal cannot be cast to bool

In [ ]:
len(df)

In [ ]:
greatdf = pd.read_parquet('all2.parquet', engine='fastparquet')

In [ ]:
type(greatdf)

# Get coordinates for Tweets (Geocoding)
- Define one spatial identifier for every user (Coordinate, Place, User Location)
- Safe user and place in a separate list
- 
- Geocode each unique location

- Slight noise for tweets at same city?



# Visualize
--> Make a time based visualization for each Innovation

- Kepler.GL?